### Задача

По заказу кредитного отдела банка необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга**.

### Оглавление

[1. Открытие и изучение данных](#1)  
[2. Предобработка данных](#2)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Обработка пропусков](#2.1)   
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Замена типа данных](#2.2)   
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Обработка дубликатов](#2.3)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 Лемматизация](#2.4)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.5 Категоризация данных](#2.5)  
[3. Ответы на вопросы](#3)  
[4. Общий вывод](#4)

### 1. Открытие и изучение данных<a id='1'></a> 

In [1]:
import pandas as pd
impo
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В DataFrame содержится 12 колонок и 21525 строк.  
Столбцы с трудовым стажем и доходом в месяц имеют NaN значения.  
Также столбец с трудовым стажем имеет нецелочисленные и отрицательные значения.  
В столбце образование информация дана с разными регистрами.

### 2. Предобработка данных <a id='2'></a>

#### 2.1 Обработка пропусков<a id='2.1'></a> 

Приведем данные из колонки с образованием к единообразию.

In [2]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [3]:
data_nan = data[
    (data['days_employed'].isnull()) & 
    (data['total_income'].isnull())]

data_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


#### Вывод

2174 строки имеют сразу два стобца без данных: трудовой стаж и доход в месяц.  
Данные могут быть пропущены в результате технической неполадки.  
Оба столбца содержат количественные переменные. 

Изучим столбец с типом занятости (income_type)

In [4]:
print('Данные для таблицы только с пропущенными значениями')
print(pd.concat([data_nan['income_type'].value_counts(), data_nan['income_type'].value_counts(normalize=True)], axis=1))
print()
print('Данные для всей таблицы')
print(pd.concat([data['income_type'].value_counts(), data['income_type'].value_counts(normalize=True)], axis=1))

Данные для таблицы только с пропущенными значениями
                 income_type  income_type
сотрудник               1105     0.508280
компаньон                508     0.233671
пенсионер                413     0.189972
госслужащий              147     0.067617
предприниматель            1     0.000460

Данные для всей таблицы
                 income_type  income_type
сотрудник              11119     0.516562
компаньон               5085     0.236237
пенсионер               3856     0.179141
госслужащий             1459     0.067782
безработный                2     0.000093
предприниматель            2     0.000093
студент                    1     0.000046
в декрете                  1     0.000046


Так как распределения частот похожи, можем установить, что пропуски случайны.
Для сотрудников, компаньонов и пенсионеров достаточно данных, чтобы рассчитать характерные значения для пропусков. Выберем медиану, чтобы оценить выборку без влияния выбросов.  
Предпринимателей всего 2 из 21525. Так как данных для заполнения не хватает, и удаление 1 строки не окажет существенного влияния на результаты, то уберем эту строку.

Удаление строки с значением в income_type - предприниматель, в total_income - NaN

In [5]:
index_entrepreneur = data_nan[
    data_nan['income_type'] == 'предприниматель'].index
data_nan = data_nan.drop(index=index_entrepreneur).reset_index(drop=True)

data = data.drop(index=index_entrepreneur).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19351 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        19351 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
print('NaN значений:', data['total_income'].isna().sum())

NaN значений: 2173


Запишем цикл для расчета значения медианы

In [7]:
unique_values_education = data_nan['education'].unique()
unique_values_income_type = data_nan['income_type'].unique()

list_median_income = []

for income_type in unique_values_income_type:
    median_income = []
    for education in unique_values_education:
        mediana = data[
            (data['income_type'] == income_type) & 
            (data['education'] == education)
        ]['total_income'].median()
        
        median_income.append(mediana)
    list_median_income.append(median_income)

Сформируем сводную таблицу с медианами месячного дохода

In [8]:
data_values = pd.DataFrame(data=list_median_income, columns=unique_values_education)

data_pivot_median_income = data_values.pivot_table(
    index=unique_values_income_type, 
    values=unique_values_education)

data_pivot_median_income

,высшее,начальное,неоконченное высшее,среднее
госслужащий,172511.107016,148339.290825,160592.345303,136652.970357
компаньон,201785.400018,136798.905143,179867.152890,159070.690289
пенсионер,144240.768611,102598.653164,120136.896353,114842.854099
сотрудник,165640.744634,125994.910603,151308.937846,136555.108821


Заменим в таблице с данными data значения NaN в столбце месячного дохода на рассчитанные медианы

In [9]:
data_final = data.copy()
for income_type in unique_values_income_type:
    for education in unique_values_education:
        data_final['total_income'].update(data_final[
            (data_final['income_type'] == income_type) & 
            (data_final['education'] == education)
        ]['total_income'].fillna(data_pivot_median_income.loc[income_type,education]))
        
data_final.info()          

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19351 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Изучим данные в столбце с трудовым стажем  
Найдем отрицательные и аномальные (больше 70 лет рабочего стажа) значения

In [10]:
MAX_DAYS_EMPLOYED = 70 * 365

print('Отрицательных значений: ', 
      data[data['days_employed'] < 0]['days_employed'].count())
print('Аномальных значений среди отрицательных: ', 
      data_final[data_final['days_employed'] < -MAX_DAYS_EMPLOYED]['days_employed'].count())

Отрицательных значений:  15906
Аномальных значений среди отрицательных:  0


Исправим отрицательные значения, которые могли появиться в результате неправильной выгрузки 

In [11]:
data_final['days_employed'].update(data_final['days_employed'].abs())
print('Отрицательных значений:', data_final[data_final['days_employed'] < 0]['days_employed'].count())

Отрицательных значений: 0


In [12]:
data_abnormal = data_final[data_final['days_employed'] > MAX_DAYS_EMPLOYED]

print('Аномальных значений:', data_abnormal['days_employed'].count())
print('{:.2%}'.format(data_abnormal['days_employed'].count() / len(data_final))) 

Аномальных значений: 3445
16.01%


Для исправления аномальных значений проверим столбец с возрастом

In [13]:
print(data_abnormal['dob_years'].unique())
print('Строки с аномальными значениями трудового стажа и возрастом равным нулю:',
      data_abnormal[data_abnormal['dob_years'] == 0]['dob_years'].count())

[53 57 67 62 68 63 64 61 54  0 65 59 60 71 38 58 55 73 52 66 56 44 69 51
 49 46 50 41 48 42 72 22 37 70 39 45 47 74 43 31 34 33 36 32 40 35 27 26
 28]
Строки с аномальными значениями трудового стажа и возрастом равным нулю: 17


17 строк с аномальными значениями трудового стажа и возрастом равным 0 не можем использовать в расчетах.
Таких строк меньше 1% от всей выборки поэтому их удаление не существенно повлияет на результаты.

In [14]:
print(data_final.groupby('dob_years')['dob_years'].count().head(2))

index_abnormal_dob_years = data_abnormal[data_abnormal['dob_years'] == 0].index

data_final.drop(index_abnormal_dob_years, inplace=True)
data_final.reset_index(drop=True,inplace=True)
print()
print(data_final.groupby('dob_years')['dob_years'].count().head(2))

dob_years
0     101
19     14
Name: dob_years, dtype: int64

dob_years
0     84
19    14
Name: dob_years, dtype: int64


Далее в работе с аномальными значениями рассчитаем медианы значений трудового стажа по десятилетиям. Для этого создаем словарь, где ключом будет являться возрастной диапазон, а значением - медиана.

In [15]:
ages = [18,30,40,50,60,70,80]
dict_mediana_days_employed = {}

for index, el in enumerate(ages):
    try:
        next_el = ages[index + 1] - 1
    except IndexError: 
        break
        
    mediana = data_final[
        (data_final['dob_years'] >= el ) & 
        (data_final['dob_years'] <= next_el) & 
        (data_final['days_employed'] < MAX_DAYS_EMPLOYED)
    ]['days_employed'].median()
    
    dict_mediana_days_employed[(el,next_el)] = mediana
    
dict_mediana_days_employed

{(18, 29): 996.7615438477428,
 (30, 39): 1589.7814009573162,
 (40, 49): 2020.0958928232772,
 (50, 59): 2261.8630175020085,
 (60, 69): 2669.073964536989,
 (70, 79): 2680.232790836156}

Заменим аномальные значения на рассчитанные медианы

In [16]:
data_conditional = data_final[data_final['days_employed'] > MAX_DAYS_EMPLOYED]

for min_age, max_age  in dict_mediana_days_employed.keys():
    dict_value = dict_mediana_days_employed.get((min_age, max_age))
    
    data_conditional_with_age = data_conditional[
        (data_conditional['dob_years'] >= min_age) & 
        (data_conditional['dob_years'] <= max_age)]
    
    data_conditional_with_age['days_employed'] = dict_value
    
    data_final['days_employed'].update(data_conditional_with_age['days_employed'])
    
print('Аномальных значений:',data_final[data_final['days_employed'] > MAX_DAYS_EMPLOYED]['days_employed'].count())

Аномальных значений: 0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Изучим столбец со стажем работы на предмет NaN значений

In [17]:
print('NaN значений:', data_final['days_employed'].isna().sum())
print('{:.2%}'.format(data_final['days_employed'].isna().sum() / len(data_final))) 

NaN значений: 2173
10.10%


Заменим NaN значения на рассчитанные медианы

In [18]:
data_final.head()
for min_age, max_age  in dict_mediana_days_employed.keys():
    dict_value = dict_mediana_days_employed.get((min_age,max_age))
    
    data_conditional_with_age = data_final[
        (data_final['dob_years'] >= min_age) & 
        (data_final['dob_years'] <= max_age)]
    
    data_conditional_with_age['days_employed'] = dict_value
    
    data_final['days_employed'].update(data_conditional_with_age['days_employed'])
    
print('NaN значений:', data_final['days_employed'].isna().sum())


NaN значений: 10


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [19]:
data_final[data_final['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1887,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,165640.744634,жилье
2280,0,NaN,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,114842.854099,недвижимость
4060,1,NaN,0,среднее,1,гражданский брак,1,M,компаньон,0,159070.690289,ремонт жилью
5009,0,NaN,0,среднее,1,женат / замужем,0,F,компаньон,0,159070.690289,покупка недвижимости
6405,0,NaN,0,высшее,0,гражданский брак,1,F,пенсионер,0,144240.768611,свадьба
6664,0,NaN,0,высшее,0,в разводе,3,F,пенсионер,0,144240.768611,покупка жилой недвижимости
8566,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,136555.108821,недвижимость
12393,3,NaN,0,среднее,1,женат / замужем,0,M,сотрудник,0,136555.108821,операции с коммерческой недвижимостью
13730,0,NaN,0,среднее,1,гражданский брак,1,F,сотрудник,0,136555.108821,на проведение свадьбы
19813,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,136555.108821,жилье


Так как возраст в 10 строках равен 0, не можем восстановить значения трудового стажа. Удалим эти строки.

In [20]:
index_nan = data_final[data_final['days_employed'].isnull()].index

data_final.drop(index_nan, inplace=True)
data_final.reset_index(drop=True, inplace=True)

data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21497 entries, 0 to 21496
Data columns (total 12 columns):
children            21497 non-null int64
days_employed       21497 non-null float64
dob_years           21497 non-null int64
education           21497 non-null object
education_id        21497 non-null int64
family_status       21497 non-null object
family_status_id    21497 non-null int64
gender              21497 non-null object
income_type         21497 non-null object
debt                21497 non-null int64
total_income        21497 non-null float64
purpose             21497 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проверим столбец с количеством детей (children)

In [21]:
data_final['children'].value_counts()

 0     14123
 1      4817
 2      2055
 3       329
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Выбросы 20 детей и -1 могли возникнуть в результате копирования, переноса данных.  
Значение 20 Не будем его учитывать при анализе данных.  
Значение -1 исправим на корректное значение.

In [22]:
data_final['children'] = data_final['children'].abs()
data_final['children'].value_counts()

0     14123
1      4864
2      2055
3       329
20       76
4        41
5         9
Name: children, dtype: int64

Проверим столбец в семейным положением (family_status)

In [23]:
data_final['family_status'].value_counts()

женат / замужем          12367
гражданский брак          4174
Не женат / не замужем     2808
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Исправим регистр

In [24]:
data_final['family_status'] = data_final['family_status'].str.lower()
data_final['family_status'].value_counts()

женат / замужем          12367
гражданский брак          4174
не женат / не замужем     2808
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Проверим столбец с задолженностями по кредиту (debt)

In [25]:
data_final['debt'].value_counts()

0    19757
1     1740
Name: debt, dtype: int64

#### Вывод

In [26]:
print('Удаленных строк: {:.2%}'.format((21525 - 21497) / 21525))  
print('Замененных строк с месячным доходом: {:.2%}'.format(5591 / 21497))     
print('Замененных строк с трудовым стажем: {:.2%}'.format(2173 / 21497))

Удаленных строк: 0.13%
Замененных строк с месячным доходом: 26.01%
Замененных строк с трудовым стажем: 10.11%


#### 2.2 Замена типа данных<a id='2.2'></a> 

In [27]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21497 entries, 0 to 21496
Data columns (total 12 columns):
children            21497 non-null int64
days_employed       21497 non-null float64
dob_years           21497 non-null int64
education           21497 non-null object
education_id        21497 non-null int64
family_status       21497 non-null object
family_status_id    21497 non-null int64
gender              21497 non-null object
income_type         21497 non-null object
debt                21497 non-null int64
total_income        21497 non-null float64
purpose             21497 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Изменим тип данных трудового стажа(days_employed) и месячного дохода(total_income) с float64 на int64

In [28]:
data_final['days_employed'] = data_final['days_employed'].astype('int64')
data_final['total_income'] = data_final['total_income'].astype('int64')
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21497 entries, 0 to 21496
Data columns (total 12 columns):
children            21497 non-null int64
days_employed       21497 non-null int64
dob_years           21497 non-null int64
education           21497 non-null object
education_id        21497 non-null int64
family_status       21497 non-null object
family_status_id    21497 non-null int64
gender              21497 non-null object
income_type         21497 non-null object
debt                21497 non-null int64
total_income        21497 non-null int64
purpose             21497 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

При переводе значений из годов в дни в первом случае и к меньшим денежным единицам во втором могли образоваться значения с типом данных float64, но для анализа данных в данном случае дробные части не нужны поэтому преобразовали в int64

#### 2.3 Обработка дубликатов<a id='2.3'></a> 

Изучим таблицу (data_final) на предмет наличия дубликатов

In [29]:
data_final.duplicated().sum()

71

Удалим дубликаты

In [30]:
data_final.drop_duplicates(inplace=True)
data_final.reset_index(drop=True, inplace=True)
data_final.duplicated().sum()

0

#### Вывод

Было найдено дубликатов 71. Они могли появиться при объединении столбцов, либо это разные записи, но без id клиента нельзя сказать точно. По имеющейся информации это дубликаты поэтому удалили их.

#### 2.4 Лемматизация<a id='2.4'></a> 

Создадим список с целями (list_purpose)

In [31]:
from pymystem3 import Mystem
m = Mystem()
list_purpose = []

for text in data_final['purpose']:
    lemmas = m.lemmatize(text)
    list_purpose.extend(lemmas)

Посчитаем как часто встречаются определенные слова в формулировках целей

In [32]:
from collections import Counter

for key, value  in Counter(list_purpose).most_common():
    print(key, value)


  33540

 21426
недвижимость 6341
покупка 5889
жилье 4453
автомобиль 4303
образование 4007
с 2916
операция 2602
свадьба 2322
свой 2228
на 2220
строительство 1877
высокий 1371
получение 1313
коммерческий 1309
для 1289
жилой 1228
сделка 941
дополнительный 905
заниматься 903
проведение 767
сыграть 765
сдача 651
семья 638
собственный 634
со 627
ремонт 606
подержанный 486
подержать 478
приобретение 461
профильный 436


Создадим список с самыми частыми целями

In [33]:
basic_purpose = ['ремонт', 'строительство', 'свадьба', 'образование', 'автомобиль', 'недвижимость', 'жилье']

#### Вывод

В основном кредиты оформляют на ремонт, строительство, свадьбу, образование, автомобиль и недвижимость.

#### 2.5 Категоризация данных<a id='2.5'></a> 

Создадим список, в котором запишем основные цели для каждой записи data_final

In [34]:
purpose_list = []

for row in data_final['purpose']:
    lemmas = m.lemmatize(row)
    for el in basic_purpose:
        if el in lemmas:
            purpose_list.append(el)
            break

Сформируем Series из этого списка и добавим в data_final как отдельную колонку

In [35]:
purpose_lem = pd.Series(purpose_list)
purpose_lem

0               жилье
1          автомобиль
2               жилье
3         образование
4             свадьба
             ...     
21421           жилье
21422      автомобиль
21423    недвижимость
21424      автомобиль
21425      автомобиль
Length: 21426, dtype: object

In [36]:
data_final.insert(11, 'base_purpose', purpose_lem)
data_final.tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,base_purpose,purpose
21421,1,2020,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,жилье,операции с жильем
21422,0,2669,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль,сделка с автомобилем
21423,1,1589,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21424,3,1589,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль,на покупку своего автомобиля
21425,2,2020,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,автомобиль,на покупку автомобиля


Создадим DataFrame, который будем дальше анализировать

In [37]:
purpose_dict = data_final[['children', 'family_status','total_income', 'debt', 'base_purpose']]
print(purpose_dict.head(10))

   children     family_status  total_income  debt base_purpose
0         1   женат / замужем        253875     0        жилье
1         1   женат / замужем        112080     0   автомобиль
2         0   женат / замужем        145885     0        жилье
3         3   женат / замужем        267628     0  образование
4         0  гражданский брак        158616     0      свадьба
5         0  гражданский брак        255763     0        жилье
6         0   женат / замужем        240525     0        жилье
7         0   женат / замужем        135823     0  образование
8         2  гражданский брак         95856     0      свадьба
9         0   женат / замужем        144425     0        жилье


#### Вывод

Категоризация целей кредита позволит в дальнейшем ответить на вопрос о зависимости цели и возврата кредита в срок.  
base_purpose содержит основные категории, выделенные после лемматизации.

### 3. Ответы на вопросы<a id='3'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
number_of_debtors_without_children = purpose_dict[
    (purpose_dict['children'] == 0) & 
    (purpose_dict['debt'] == 1)]['debt'].count()

number_of_people_without_children = purpose_dict[
    purpose_dict['children'] == 0]['debt'].count()

percent_number_of_debtors_without_children = (
    number_of_debtors_without_children / number_of_people_without_children * 100)

number_of_debtors_with_children = purpose_dict[
    (purpose_dict['children'] > 0) & 
    (purpose_dict['debt'] == 1)]['debt'].count()

number_of_people_with_children = purpose_dict[
    purpose_dict['children'] > 0]['debt'].count()

percent_number_of_debtors_with_children = (
    number_of_debtors_with_children / number_of_people_with_children * 100)

table_children = pd.DataFrame(data=[
    ['нет',
    number_of_people_without_children,
    number_of_debtors_without_children, 
    percent_number_of_debtors_without_children],
    ['есть',
    number_of_people_with_children,
    number_of_debtors_with_children, 
    percent_number_of_debtors_with_children]], columns =[
    'Наличие детей', 'Всего людей', 'Всего должников', 'Процент должников'])

print(table_children)

  Наличие детей  Всего людей  Всего должников  Процент должников
0           нет        14065             1062           7.550658
1          есть         7361              678           9.210705


In [39]:
total_people = purpose_dict.groupby(purpose_dict['children'] > 0)['debt'].count()
debtors = purpose_dict[purpose_dict['debt'] == 1].groupby(purpose_dict['children'] > 0)['debt'].count()
pd.concat([total_people, debtors, debtors / total_people * 100], axis=1, keys=['Всего', 'Всего должников', 'Процент должников'])

,Всего,Всего должников,Процент должников
children,,,
False,14065,1062,7.550658
True,7361,678,9.210705


### Вывод

По имеющемся данным можно выделить зависимость наличия детей и возврата кредита в срок. Люди без детей имеют задолженность в 7,55% случаях, в то время как люди с детьми в 9,21%. Возможная причина - у людей с детьми может быть больше незапланированных трат.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим функцию, которая будет считать основные показатели при нужном условии:  
-количество людей всего,  
-количество людей, имеющих задолженность  
-процент людей, имеющих задолженность, от всего количества людей

In [40]:
def create_list(column):
    list_base_indicator = []
    for el in purpose_dict[column].unique():
        number_all = purpose_dict[
        purpose_dict[column] == el]['debt'].count()
    
        number_of_debtors = purpose_dict[
        (purpose_dict[column] == el) & 
        (purpose_dict['debt'] == 1)]['debt'].count()
    
        percent_number = (
        number_of_debtors / number_all * 100)
        list_base_indicator.append([el, number_all, number_of_debtors, percent_number])
        
    return list_base_indicator

In [41]:
list_base_indicator = create_list('family_status')       
table_family_status = pd.DataFrame(data=list_base_indicator, columns =[
    'Семейное положение', 'Всего людей', 'Всего должников', 'Процент должников']) 
print(table_family_status)

      Семейное положение  Всего людей  Всего должников  Процент должников
0        женат / замужем        12326              931           7.553140
1       гражданский брак         4148              388           9.353905
2         вдовец / вдова          954               62           6.498952
3              в разводе         1193               85           7.124895
4  не женат / не замужем         2805              274           9.768271


### Вывод

В данном случае наблюдается,что показатели 6-7% у людей, кто находится или был в браке, а показатели 9% у людей свободных или в гражданских отношениях. Возможно, в первом случе люди старше или уже научились брать ответственность и могут распланировать возврат долга в нужные сроки.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Добавим колонку с диапазонами доходов

In [42]:
import numpy as np
purpose_dict['ranges_total_income'] = pd.cut(purpose_dict['total_income'],[
    0,100000,135000,165000,220000, np.inf])
purpose_dict.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,children,family_status,total_income,debt,base_purpose,ranges_total_income
0,1,женат / замужем,253875,0,жилье,"(220000.0, inf]"
1,1,женат / замужем,112080,0,автомобиль,"(100000.0, 135000.0]"
2,0,женат / замужем,145885,0,жилье,"(135000.0, 165000.0]"
3,3,женат / замужем,267628,0,образование,"(220000.0, inf]"
4,0,гражданский брак,158616,0,свадьба,"(135000.0, 165000.0]"


In [43]:
list_base_indicator = create_list('ranges_total_income') 

table_total_income = pd.DataFrame(data=list_base_indicator, columns =[
    'Диапазон дохода', 'Всего людей', 'Всего должников', 'Процент должников']) 
print(table_total_income.sort_values(by='Всего людей', ascending=False))

        Диапазон дохода  Всего людей  Всего должников  Процент должников
3       (0.0, 100000.0]         4455              354           7.946128
1  (100000.0, 135000.0]         4414              373           8.450385
2  (135000.0, 165000.0]         4359              393           9.015829
4  (165000.0, 220000.0]         4194              336           8.011445
0       (220000.0, inf]         4004              284           7.092907


### Вывод

Данная зависимость похожа на нормальную. В диапазонах с низким и высоким доходами задолженностей меньше (7%), чем в среднем диапазоне от 135000 до 165000 (9%). Чем ближе к среднему диапазону месячный доход, тем больше процент задолженности.

- Как разные цели кредита влияют на его возврат в срок?

In [44]:
list_base_indicator = create_list('base_purpose') 

table_total_income = pd.DataFrame(data=list_base_indicator, columns =[
    'Причина кредитования', 'Всего людей', 'Всего должников', 'Процент должников'])
print(table_total_income)

  Причина кредитования  Всего людей  Всего должников  Процент должников
0                жилье         3847              273           7.096439
1           автомобиль         4303              402           9.342319
2          образование         4007              370           9.233841
3              свадьба         2322              186           8.010336
4         недвижимость         4464              330           7.392473
5        строительство         1877              144           7.671817
6               ремонт          606               35           5.775578


In [45]:
purpose_dict.pivot_table(
    index='base_purpose', 
    values='debt', aggfunc=['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:,.2%}'.format})

,count,sum,mean
,debt,debt,debt
base_purpose,,,
автомобиль,4303,402,9.34%
жилье,3847,273,7.10%
недвижимость,4464,330,7.39%
образование,4007,370,9.23%
ремонт,606,35,5.78%
свадьба,2322,186,8.01%
строительство,1877,144,7.67%


### Вывод

Самый низкий процент задолженности у кредитов на ремонт, но размер выборки сильно отличается от остальных. Тем не менее, у людей с такой целью кредитования скорее всего уже есть своя недвижимость, они более основательны. А вот люди с целью кредитования автомобиль или образование могут быть менее стабильными в финансовом плане из-за влияния других факторов(возраст, наличие работы и др.). То есть зависимость есть, но косвенная.  

### 4. Общий вывод <a id='4'></a>

Люди без детей имеют задолженность почти на 2% меньше, чем с детьми.  
Люди, которые находятся или были в браке, имеют задолженность на 2% меньше, чем у людей свободных или в гражданских отношениях.  
С уровнем дохода видно нормальное распределение, где купол - это средний доход от 135 тыс. до 165 тыс.  
Разные цели кредитования имеют разный процент задолженности. Есть зависимость в пределах 3%.